In [8]:
import pandas as pd
import yfinance as yf

In [ ]:
eth = yf.Ticker('ETH-USD')
eth_data = eth.history('10y')

In [17]:
int(10_000 / (2_290 / 90))

393

In [22]:
buy_interval = 90
price = eth_data['Close']
eth_data['buy_in'] = int(10_000 / (price.count() / buy_interval))

In [52]:
data = eth_data.iloc[::90].drop(['Open', 'Low', 'Volume', 'Dividends', 'Stock Splits', 'High'], axis=1)
data['eth_bought'] = data['buy_in'] / data['Close']
data

,Close,buy_in,eth_bought
Date,,,
2017-11-09 00:00:00+00:00,320.884003,393,1.224742
2018-02-07 00:00:00+00:00,757.067993,393,0.519108
2018-05-08 00:00:00+00:00,752.856995,393,0.522011
2018-08-06 00:00:00+00:00,406.657990,393,0.966414
2018-11-04 00:00:00+00:00,207.485992,393,1.894104
2019-02-02 00:00:00+00:00,110.430260,393,3.558807
2019-05-03 00:00:00+00:00,167.952408,393,2.339949
2019-08-01 00:00:00+00:00,217.808441,393,1.804338
2019-10-30 00:00:00+00:00,184.692169,393,2.127865


In [60]:
total_eth_bought = data['eth_bought'].sum()
usd_value = total_eth_bought * eth_data['Close'].iloc[-1]

In [62]:
usd_value / 10_000 * 100

684.0222637876933